Moving onto the tensorflow classification

In [148]:
import pandas as pd
from sklearn.model_selection import train_test_split
replacements = [",", ".", ":", "'",'\'', '—', '"', '\'' "!", "?", "." ";", "@", "{", "}", "[", "]", "(", ")", "/", "#", "*", "_", "%", "-", "£", "$", "&"]
articles_prepositions = ['a', 'an', 'the', 'about', 'above', 'across', 'after', 'against', 'among', 'around',
'at', 'before', 'behind', 'below', 'beside', 'between', 'by', 'down', 'during', 'for', 'from', 'in', 'inside',
'into', 'near', 'of', 'off', 'on', 'out', 'over', 'through', 'to', 'toward', 'under', 'up', 'with']
texts = []
biases = []
data = {}
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()
for i in range(0,2691):
    f = open("articles/article%d.txt" % i, 'r', encoding="utf-8")
    contents = f.read().lower()
    f.close()

    contents = contents.replace("cnn", '')
    contents = contents.replace("cnn", '')
    contents = contents.replace("advertisement", '')
    contents = contents.replace("fox", '')
    contents = contents.replace("huffpost", '')
    contents = contents.replace("washingtonpost", '')
    contents = contents.replace("washington post", '')
    contents = contents.replace("the federalist", '')
    contents = contents.replace("\n", '')
    for r in range(0, len(replacements)):
        contents = contents.replace(replacements[r], '')

    for ap in range(0, len(articles_prepositions)):
        contents = contents.replace((' ' +articles_prepositions[ap] + ' '), ' ')
    
    stuff = contents.split()
    filtered_stuff = [word for word in stuff if word not in stopwords.words('english')]
    contents = ''
    for word in filtered_stuff:
        word = ps.stem(word)
        contents = contents + word + ' '

    texts.append(contents)
    
    g = open("articleBiases/article%dBias.txt" % i, 'r', encoding='utf-8')
    bias = g.read()
    g.close()
    biases.append(bias)

data = {'Article': texts, 'Bias': biases}

df = pd.DataFrame(data)

df = df.dropna()
df.isnull().sum()

df



,Article,Bias
0,arizona republican gubernatori candid kari lak...,0
1,washington former vice presid mike penc wednes...,0
2,modern republican party’ one nonnegoti valu ru...,0
3,editor’ note elizabeth alexand poet scholar pr...,0
4,editor’ note van jone host polit comment found...,0
...,...,...
2686,amsterdam’ worldfam redlight district could lo...,1
2687,crime risen major citi nationwid recent year m...,1
2688,first nonprofit liber billionair georg soro op...,1
2689,new listen news articles!twitt ceo elon musk g...,1


In [174]:
gf = pd.read_csv('finalDataset')
gf

,Unnamed: 0,Article,Bias
0,0,arizona republican gubernatori candid kari lak...,0
1,1,washington former vice presid mike penc wednes...,0
2,2,modern republican party’ one nonnegoti valu ru...,0
3,3,editor’ note elizabeth alexand poet scholar pr...,0
4,4,editor’ note van jone host polit comment found...,0
...,...,...,...
2686,2686,amsterdam’ worldfam redlight district could lo...,1
2687,2687,crime risen major citi nationwid recent year m...,1
2688,2688,first nonprofit liber billionair georg soro op...,1
2689,2689,new listen news articles!twitt ceo elon musk g...,1


In [172]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from sklearn.utils import compute_class_weight
opt = tf.optimizers.Adam(learning_rate = 0.001)
#opt = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
tokenizer = Tokenizer(oov_token="<OOV>")


#splitting data into a training and test dataset
split = round(len(df)*0.8)
train_reviews = df['Article'][:split]
train_label = df['Bias'][:split]
test_reviews = df['Article'][split:]
test_label = df['Bias'][split:]

#sanitizing all data as a string
import numpy as np
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for row in train_reviews:
    training_sentences.append(str(row))
for row in train_label:
    training_labels.append(int(row))
for row in test_reviews:
    testing_sentences.append(str(row))
for row in test_label:
    testing_labels.append(int(row))

# variable constants
vocab_size = 40000
embedding_dim = 16
max_length =  1500
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

# tokenizing words
#tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

    # sentences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # usually essential/first layer
    tf.keras.layers.GlobalAveragePooling1D(),
    #tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    #tf.keras.layers.Dense(12, activation='tanh'),
    #tf.keras.layers.Dense(12, activation='softmax'), # don't use softmax for output layer
    tf.keras.layers.Dense(1, activation='sigmoid'),
    #tf.keras.layers.Dense(1, activation='softmax')
])

#model.add(BatchNormalization())
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

#training 

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
#class_weight = {1:0.5,1:1.}
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)

num_epochs = 20
#history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final), shuffle=True, callbacks=[checkpoint1])
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final), shuffle=True)

prediction1 = model.predict(testing_padded[:20])
print(prediction1)
print(test_label[:20])

Model: "sequential_160"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_158 (Embedding)   (None, 1500, 16)          640000    
                                                                 
 global_average_pooling1d_98  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_343 (Dense)           (None, 16)                272       
                                                                 
 dense_344 (Dense)           (None, 1)                 17        
                                                                 
Total params: 640,289
Trainable params: 640,289
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
68/68 [==============================] - 2s 13ms/step - loss: 0.6792 - accuracy: 0.6224 - val_loss: 0

C:\Users\markv\AppData\Local\Temp\ipykernel_18576\1079722132.py:92: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  print(test_label[:20])


In [ ]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
replacements = [",", ".", ":", '\'','\'', '"', "!", "?", "." ";", "@", "{", "}", "[", "]", "(", ")", "/", "#", "*", "_", "%", "-", "£", "$", "&"]
articles_prepositions = ['a', 'an', 'the', 'about', 'above', 'across', 'after', 'against', 'among', 'around',
'at', 'before', 'behind', 'below', 'beside', 'between', 'by', 'down', 'during', 'for', 'from', 'in', 'inside',
'into', 'near', 'of', 'off', 'on', 'out', 'over', 'through', 'to', 'toward', 'under', 'up', 'with']
articles = []
details = []
biases = []
for i in range(0,300):
        words = []
        counts = []
        isPunctuation = []
        articles.append('Article %d' % i)
        # Opening the article file, assigning text it's contents, and closing the file
        f = open("cleanedArticles/Article%d.txt" % i, 'r', encoding="utf-8")
        lines = f.readlines()
        f.close()

# reading the sanitized data from Article0 and turning it into a pandas dataframe
        for line in lines:
                data = line.split()
                newWord = data[0]
                newWord = newWord[0:(len(newWord)-1)]
                
                for exempt in articles_prepositions:
                        if(newWord==exempt):
                                continue

                if((len(newWord)==3) and (newWord[1:2] in letters)):
                        continue
                for r in range(0, len(replacements)):
                        if(len(newWord) > 3): 
                                newWord = newWord.replace(replacements[r], "")
                        
                for r in range(0, len(replacements)):
                        data[1] = data[1].replace(replacements[r], "")
                if(len(newWord)>2):
                        if(newWord[(len(newWord)-1):(len(newWord))] == 'i'):
                                newWord = newWord[0:(len(newWord)-1)]

                words.append(newWord)
                counts.append(data[1])
        g=0
        #implementing the isPunctuation functionality
        for word in words:
                isPunctuation.append('')

                if(len(word) == 3):
                        for letter in letters:
                                if(word.__contains__(letter)):
                                        isPunctuation[g] = ('False')
                        if(isPunctuation[g] != 'False'):
                                isPunctuation[g] = 'True'
                if(isPunctuation[g] == ''):
                        isPunctuation[g] = 'False'
                g=g+1
        newWords = []
        g=0
        for word in words:
                if(isPunctuation[g] == 'True'):
                        word = word[1:2]
                newWords.append(word)
                g+=1
        
        # storing every sentence in an article as a list
        sentences = []
        text = ""

        f = open("articles/article%d.txt" % i, 'r', encoding="utf-8")
        text = str(f.read()).lower()
        f.close()
        closingPunctuation = ['.', '?', '!']

        text = text.replace("\n", "")

        for r in range(0, len(closingPunctuation)):
            text = text.replace((closingPunctuation[r]), closingPunctuation[r] + "|")
            text = text.replace("| ", "|")

        sentences = text.split('|')

        mapping = []
        map = {}
        for word in newWords:
                sentenceIndex=0
                sentenceList = ""
                for sentence in sentences:
                        if(sentence.__contains__(word)):
                                sentenceList = sentenceList + str(sentenceIndex) + " "
                        sentenceIndex+=1
                sentenceList = sentenceList[0:(len(sentenceList)-1)]
                mapping.append(sentenceList)

        #finding words that are used in the same sentences
        sentences = []
        text = ""
        f = open("articles/article0.txt", 'r', encoding="utf-8")
        text = str(f.read()).lower()
        f.close()
        closingPunctuation = ['.', '?', '!']

        text = text.replace("\n", "")

        for r in range(0, len(closingPunctuation)):
                text = text.replace((closingPunctuation[r]), closingPunctuation[r] + "|")
                text = text.replace("| ", "|")

        sentences = text.split('|')


        # it works!

        deets = []
        terms = []
        sentenceGroups = []
        sentenceInfo =[]
        terms = newWords
        #logic here is a little sloppy, could be condensed
        for term in terms:
                sentenceGroups=[]
                sentenceID = 0
                for sentence in sentences:
                        wordsInSentence= []
                        wordsTogether = []
                        if(sentence.__contains__(term)):
                                wordsInSentence = sentence.split()
                                for potentialWord in wordsInSentence:
                                        if(potentialWord!=term):
                                                for replacee in replacements:
                                                        potentialWord = potentialWord.replace(replacee, '')
                                                wordsTogether.append(potentialWord)
                        else:
                                continue
                        sentenceGroups.append(wordsTogether)
                sentenceInfo.append(sentenceGroups)
        g = open("articleBiases/article%dBias.txt" % i, 'r', encoding="utf-8")
        bias = g.read()
        g.close()
        if(bias.__contains__('-1')):
                biases.append('-1')
        else:
                biases.append('1')
        
        details.append({'Words': newWords, 'Counts':counts, 'isPunctuation': isPunctuation, 'sentenceMap': mapping, 'Associations': sentenceInfo})
        
dfnew = pd.DataFrame(details)
#dfnew['Article'] = articles
dfnew['Bias'] = biases

dfnew